In [1]:
import pandas as pd
import os

In [2]:
import scipy.io


In [3]:
data = pd.read_csv('iden_split.txt', sep=" ", header=None)
data.columns = ["labels", "file_name"]

In [4]:
data.head()

# label 1 train
# label 2 valiation
# label 3 test


,labels,file_name
0,3,id10003/na8-QEFmj44/00003.wav
1,1,id10003/tCq2LcKO6xY/00002.wav
2,1,id10003/K5zRxtXc27s/00001.wav
3,1,id10003/bDxy7bnj_bc/00004.wav
4,1,id10003/E_6MjfYr0sQ/00011.wav


In [5]:
path = '../wav/' # path for wav files

In [6]:
all_ids = os.listdir(path)

In [7]:
from collections import defaultdict
all_files = defaultdict(list)

for individual_main in all_ids:
    individual_subdir = os.listdir(path + individual_main)
    for directory in individual_subdir:
        for wav_files in os.listdir(path + individual_main + '/' + directory):
            file = directory + '/' + wav_files
            all_files[individual_main].append(directory + '/' + wav_files)


In [8]:
len(all_files)

1251

In [13]:
data.groupby('labels').first()

,file_name
labels,
1,id10003/tCq2LcKO6xY/00002.wav
2,id10003/NC70RWJDnMg/00003.wav
3,id10003/na8-QEFmj44/00003.wav


In [9]:
from collections import defaultdict
dict_of_labels = defaultdict(list)

for individual_id in all_files:
    for file in all_files[individual_id]:
        name = individual_id + '/' + file
        label = data[data['file_name'] == name]['labels']
        label = label.to_numpy()[0]

        if label == 1:
            dict_of_labels[individual_id].append("train")
        elif label == 3:
            dict_of_labels[individual_id].append("test")
        elif label == 2 :
            dict_of_labels[individual_id].append("validation")
        else:
            dict_of_labels[individual_id].append("not found")




In [10]:
all_matlab_files = os.listdir('./voiceFeatsIdentification')

In [11]:
all_matlab_files[0:5]

['id10001.mat', 'id10002.mat', 'id10003.mat', 'id10004.mat', 'id10005.mat']

In [28]:
train_data = []
train_labels = []
test_data = []
test_labels = []
validation_data = []
validation_labels = []


for mat_file in all_matlab_files:
    identity = mat_file.split('.')[0]
    mat = scipy.io.loadmat('./voiceFeatsIdentification/' + mat_file)
    if len(mat["allFeats"]) == len(dict_of_labels[identity]):
        for features, label in zip (mat['allFeats'], dict_of_labels[identity]):
            if label == "train":
                train_data.append(features[0])
                train_labels.append(identity)
#                 print(train_data[-1], train_labels[-1])
            elif label == "test":
                test_data.append(features[0])
                test_labels.append(identity)
            elif label == "validation":
                validation_data.append(features[0])
                validation_labels.append(identity)
            else:
                print("label not understood")
                
    else:
        print(len(mat['allFeats']))
        print(len(dict_of_labels[identity]))
                

    
    
    

In [29]:
len(train_data)

138361

In [30]:
len(train_labels)

138361

In [31]:
len(test_data)

8251

In [32]:
len(test_labels)

8251

In [15]:
len(dict_of_labels)

1251

In [35]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [36]:
le.fit(train_labels)

LabelEncoder()

In [38]:
le.classes_

array(['id10001', 'id10002', 'id10003', ..., 'id11249', 'id11250',
       'id11251'], dtype='<U7')

In [54]:
y_train = le.transform(train_labels)


138361

In [55]:
y_train = y_train.reshape(y_train.shape[0],1)

In [56]:
y_test = le.transform(test_labels)

In [61]:
import numpy as np
train_data =  np.array(train_data)
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1])

In [57]:
from sklearn import svm

In [66]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(train_data, y_train)

C:\Users\c2vr\Anaconda3\envs\vgg_gpu\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(decision_function_shape='ovo')

In [68]:
test_data =  np.array(test_data)
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1])

In [67]:
y_test = y_test.reshape(y_test.shape[0],1)

(8251,)

In [69]:
predictions = clf.predict(test_data)

In [70]:
from sklearn.metrics import classification_report

In [71]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         5
           3       0.78      0.88      0.82         8
           4       0.86      1.00      0.92         6
           5       0.86      1.00      0.92         6
           6       0.75      1.00      0.86         6
           7       0.83      1.00      0.91         5
           8       1.00      0.71      0.83         7
           9       1.00      0.83      0.91         6
          10       0.56      1.00      0.71         5
          11       1.00      1.00      1.00         6
          12       1.00      0.80      0.89         5
          13       0.86      1.00      0.92         6
          14       1.00      0.83      0.91         6
          15       0.88      1.00      0.93         7
          16       1.00      1.00      1.00         6
          17       0.88    

C:\Users\c2vr\Anaconda3\envs\vgg_gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:


######   Kindly keep this tab open
                                        Thank you.


